<a href="https://colab.research.google.com/github/suinkangme/comp433_project/blob/main/COMP433_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing a robust CNN model to address the challenge of learning with label noise in  CIFAR10 dataset

- CIFAR10 Label : ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’.

- image size : 3x32x32




In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os


## Load and normalize CIFAR10

- Transform them to Tensors of normalized range [-1, 1].

In [2]:
transform = transforms.Compose(
                              [transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
train_data = torchvision.datasets.CIFAR10(root='./data',
                                          train=True,
                                          download=True,
                                          transform = transform)

test_data = torchvision.datasets.CIFAR10(root='./data',
                                          train=False,
                                          download=True,
                                          transform = transform)

100%|██████████| 170498071/170498071 [00:13<00:00, 12313752.21it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
train_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=2)

test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=2)

## Define & train with a  baseline CNN model

In [6]:
class BaselineModel(nn.Module):
  def __init__(self):
    super(BaselineModel,self).__init__()
    self.features = nn.Sequential(
      nn.Conv2d(3, 8, kernel_size=3, padding = 1),  # (input channel, output channels, kernel size, padding)  32*32*8
      nn.ReLU(inplace=True), # activation function modifies the input tensor directly
      nn.Conv2d(8, 16, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2,stride=2), # 16*16*16

      nn.Conv2d(16, 32, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.Conv2d(32, 64, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.MaxPool2d(kernel_size=2,stride=2) # 8*8*128
    )

    # fully connected layers
    self.fc_layers = nn.Sequential(
      nn.Linear(128*8*8, 120),
      nn.ReLU(inplace=True),
      nn.Linear(120,84),
      nn.ReLU(inplace=True),
      nn.Linear(84,10)
    )


  def forward(self, x):
    x = self.features(x)
    x = torch.flatten(x,1)
    x = self.fc_layers(x)
    return x

In [7]:
# Training on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net = BaselineModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [8]:
net = net.to(device)
net.train()
num_epochs = 5

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Move data to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/5, Loss: 2.29799747467041
Epoch 2/5, Loss: 3.0276448726654053
Epoch 3/5, Loss: 1.5086110830307007
Epoch 4/5, Loss: 1.386398434638977
Epoch 5/5, Loss: 1.439256191253662


## Noise Labeling
- 5 different noise levels (10%,
30%, 50%, 80%, 90%)

### Symmetric label noise

In [9]:
def replace_symmetric_noise(labels, epsilon):
    num_labels = len(labels)
    num_flips = int(epsilon * num_labels)

    # choose the label to be flipped
    flip_indices = np.random.choice(num_labels, num_flips, replace=True)

    # filp the label
    labels[flip_indices] = np.random.randint(0, 10, num_flips)

    return labels

### Asymmetric label noise

In [ ]:
def flip_labels_asymmetrically(labels, epsilon):
    flip_rules = {
        9: 1,   # Truck to Automobile
        2: 0,   # Bird to Airplane
        4: 7,   # Deer to Horse
        3: 5,   # Cat to Dog
        5: 3,   # Dog to Cat
    }

    flipped_labels = []
    for label in labels:
        # Check if label flipping should occur based on epsilon
        if np.random.random() < epsilon:
            # Flip the label based on the flip_rules dictionary
            flipped_label = flip_rules.get(label, label)
            flipped_labels.append(flipped_label)
        else:
            # If no flipping, keep the original label
            flipped_labels.append(label)

    return np.array(flipped_labels)


In [ ]:
# Create a sample target array with ten '9's, ten '2's, ten '4's, five '3's and ten '5's
test_array = np.array([9] * 10 + [2] * 10 + [4] * 10 + [3] * 5 + [5]*10)

print(test_array)

In [ ]:
flip_labels_asymmetrically(test_array,1)

## Train the model - baseline

### Train with the symmetric noise labeling

In [10]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

# Create a dictionary with keys in the format 'noise_level_{100 * value}'
model_dict = {f'noise_level_{int(100 * level)}_sy': None for level in noise_levels}

# Training on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epsilon in noise_levels:

    net = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001)

    net = net.to(device)
    net.train()
    num_epochs = 5

    print(f"Symmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            # Add symmetric noise to labels
            labels_noisy = torch.from_numpy(replace_symmetric_noise(labels.numpy(), epsilon))

            # Move data to GPU
            inputs, labels_noisy = inputs.to(device), labels_noisy.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = net(inputs)
            loss = criterion(outputs, labels_noisy)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

    # save model to dictionary
    model_dict[f'noise_level_{int(100 * epsilon)}_sy'] = net.state_dict()


Symmetric Training with noise level: 0.1
Epoch 1/5, Loss: 2.3005621433258057
Epoch 2/5, Loss: 2.2650551795959473
Epoch 3/5, Loss: 2.038346529006958
Epoch 4/5, Loss: 1.4505876302719116
Epoch 5/5, Loss: 1.2883195877075195
Symmetric Training with noise level: 0.3
Epoch 1/5, Loss: 2.2960057258605957
Epoch 2/5, Loss: 2.3021602630615234
Epoch 3/5, Loss: 2.307368755340576
Epoch 4/5, Loss: 2.182286500930786
Epoch 5/5, Loss: 1.8889628648757935
Symmetric Training with noise level: 0.5
Epoch 1/5, Loss: 2.2977547645568848
Epoch 2/5, Loss: 2.3006043434143066
Epoch 3/5, Loss: 2.299471139907837
Epoch 4/5, Loss: 2.331737756729126
Epoch 5/5, Loss: 2.1763811111450195
Symmetric Training with noise level: 0.8
Epoch 1/5, Loss: 2.302483081817627
Epoch 2/5, Loss: 2.3050577640533447
Epoch 3/5, Loss: 2.308727979660034
Epoch 4/5, Loss: 2.2982776165008545
Epoch 5/5, Loss: 2.2883522510528564
Symmetric Training with noise level: 0.9
Epoch 1/5, Loss: 2.298356294631958
Epoch 2/5, Loss: 2.2972939014434814
Epoch 3/5, 

In [12]:
for key, state_dict in model_dict.items():
    net = BaselineModel()
    net.load_state_dict(state_dict)
    net.eval()
    net.to(device)

    # Variables to store predictions and ground truth labels
    num_correct_predictions = 0
    total_num_predictions = 0
    loss = 0.0

    # Iterate over the test dataset
    with torch.no_grad():  # temporarily set all requires_grad flags to False
        for i, (data, label) in enumerate(test_loader):
            # move inputs to desired device and dtype
            data = data.to(device, dtype=torch.float32)
            label = label.to(device, dtype=torch.long)

            # forward pass
            logit = net(data)

            # compute loss and number of accurate predictions
            loss += torch.nn.functional.cross_entropy(logit, label, reduction='sum').item()
            preds = logit.max(dim=1)[1]
            num_correct_predictions += (preds == label).sum().item()
            total_num_predictions += len(preds)

        # compute average loss
        loss /= total_num_predictions

        # compute accuracy percentage
        accuracy = (float(num_correct_predictions) / total_num_predictions) * 100

        print(f"Symmetric {key} Accuracy: {accuracy:.2f}%, Loss: {loss:.4f}")


Symmetric noise_level_10_sy Accuracy: 46.55%, Loss: 1.4855
Symmetric noise_level_30_sy Accuracy: 32.83%, Loss: 1.9129
Symmetric noise_level_50_sy Accuracy: 26.64%, Loss: 2.0702
Symmetric noise_level_80_sy Accuracy: 17.29%, Loss: 2.2893
Symmetric noise_level_90_sy Accuracy: 18.11%, Loss: 2.2939


### Train with the asymmetric noise labeling

In [ ]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]
# Create a dictionary with keys in the format 'noise_level_{100 * value}'
model_dict = {f'noise_level_{int(100 * level)}_asy': None for level in noise_levels}



# Training on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epsilon in noise_levels:


    net = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001)


    net = net.to(device)
    net.train()
    num_epochs = 5


    print(f"Asymmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            # Add symmetric noise to labels
            labels_noisy = torch.from_numpy(flip_labels_asymmetrically(labels.numpy(), epsilon))

            # Move data to GPU
            inputs, labels_noisy = inputs.to(device), labels_noisy.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = net(inputs)
            loss = criterion(outputs, labels_noisy)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')
    # save model to dictionary
    model_dict[f'noise_level_{int(100 * epsilon)}_asy'] = net


In [ ]:
for key, net in model_dict.items():
    net.eval()
    net.to(device)

    # Variables to store predictions and ground truth labels
    num_correct_predictions = 0
    total_num_predictions = 0

    # Iterate over the test dataset
    with torch.no_grad():  # temporarily set all requires_grad flags to False
        for i, (data, label) in enumerate(test_loader):
            ### TODO: perform forward pass and compute the loss and accuracy

            # move inputs to desired device and dtype
            data = data.to(device, dtype=torch.float32)
            label = label.to(device, dtype=torch.long)

            # forward pass
            logit = net(data)

            # compute loss and number of accurate predictions
            loss += torch.nn.functional.cross_entropy(logit, label, reduction='sum').item()
            preds = logit.max(dim=1)[1]
            num_correct_predictions += (preds == label).sum().item()
            total_num_predictions += len(preds)


        # compute accuracy percentage
        loss = loss/total_num_predictions
        accuracy = (float(num_correct_predictions) / total_num_predictions)
    print(f"Asymmetric {key} Accuracy: {accuracy * 100:.2f}%")

### Save the trained model

In [ ]:
# save the baseline model
PATH = './BaselineModel.pth'
torch.save(net.state_dict(), PATH)

## Testing